## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-22-18-51-00 +0000,nyt,Live Updates: Palestinian Statehood Will Be Fo...,https://www.nytimes.com/live/2025/09/22/world/...
1,2025-09-22-18-50-57 +0000,nyt,Zelensky Urges ‘Powerful Pressure on Russia’ a...,https://www.nytimes.com/2025/09/22/world/europ...
2,2025-09-22-18-48-48 +0000,nypost,Brute beaten by NYC vigilante after allegedly ...,https://nypost.com/2025/09/22/us-news/brute-be...
3,2025-09-22-18-47-44 +0000,nyt,What to Know About the UN’s Palestinian Stateh...,https://www.nytimes.com/live/2025/09/22/world/...
4,2025-09-22-18-44-54 +0000,nypost,Manhattan casino plan nixed after board closes...,https://nypost.com/2025/09/22/us-news/manhatta...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
108,trump,47
54,kirk,33
53,charlie,24
110,memorial,17
107,president,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-09-22-18-27-02 +0000,nypost,"Elon Musk, President Trump come together at Ch...",https://nypost.com/2025/09/22/us-news/elon-mus...,154
237,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...,148
144,2025-09-22-11-14-50 +0000,nypost,Charlie Kirk inspired Trump and Elon Musk to r...,https://nypost.com/2025/09/22/us-news/charlie-...,147
244,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk shake hands at Char...",https://nypost.com/2025/09/21/us-news/presiden...,146
229,2025-09-21-23-30-00 +0000,wsj,President Trump and the upper echelon of his a...,https://www.wsj.com/us-news/charlie-kirk-funer...,144


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,154,2025-09-22-18-27-02 +0000,nypost,"Elon Musk, President Trump come together at Ch...",https://nypost.com/2025/09/22/us-news/elon-mus...
238,44,2025-09-21-22-27-41 +0000,nypost,Erika Kirk forgives assassin who killed husban...,https://nypost.com/2025/09/21/us-news/erika-ki...
86,42,2025-09-22-15-31-23 +0000,nypost,Howard Stern blasts ABC suspension of Jimmy Ki...,https://nypost.com/2025/09/22/media/howard-ste...
161,36,2025-09-22-10-00-00 +0000,nypost,NJ governor’s race heats up as Ciattarelli’s G...,https://nypost.com/2025/09/22/us-news/the-kama...
200,36,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...
44,34,2025-09-22-17-29-04 +0000,nypost,Jill Biden’s ex-spokesman reveals ‘cult’ of ‘m...,https://nypost.com/2025/09/22/us-news/michael-...
2,33,2025-09-22-18-48-48 +0000,nypost,Brute beaten by NYC vigilante after allegedly ...,https://nypost.com/2025/09/22/us-news/brute-be...
191,33,2025-09-22-06-35-15 +0000,bbc,South Korea would accept a Trump-Kim deal to f...,https://www.bbc.com/news/articles/cwy91w0e1z2o...
11,32,2025-09-22-18-28-40 +0000,nypost,Russia risks ‘expansion’ of war with intrusion...,https://nypost.com/2025/09/22/us-news/russia-r...
47,29,2025-09-22-17-21-27 +0000,nypost,Zohran Mamdani slammed by high-profile NY rabb...,https://nypost.com/2025/09/22/us-news/zohran-m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
